In [ ]:
import cv2
import numpy as np
from time import time

class FaceCapture:
    def __init__(self, video=0, face_threshold=0.025, camht=480, camwd=640, rescale=8, noDetectionLimit=0, detectargs={'scaleFactor': 1.1,
                                                                                                                       'minNeighbors': 5, 'minSize': (30, 30),
                                                                                                                       'flags': cv2.CASCADE_SCALE_IMAGE}):
        self.face_threshold = face_threshold
        self.detectargs = detectargs
        self.camwd = camwd
        self.camht = camht
        self.cam = cv2.VideoCapture(video)
        self.cam.set(3, camwd)
        self.cam.set(4, camht)
        self.camDiag = np.sqrt(camwd*2 + camht*2)
        self.faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        self.roi = None
        self.face1 = None 
        self.noDetectionLimit = noDetectionLimit
        self.noDetectionCounter = 0

    def getCamDims(self):
        return (int(self.camwd), int(self.camht))

    def getCamShape(self):
        return (int(self.camht), int(self.camwd))

    def withinThreshold(self, loc):
        dF = np.abs(self.face1 - np.array(loc)) / self.camDiag
        return np.all(dF <= self.face_threshold)

    def retCapture(self, returnSuccess=False):
        if returnSuccess == False:
            ret_val, img = self.cam.read()
            if img.shape[:2] != (self.camht, self.camwd):
                img = cv2.resize(img, (self.camwd, self.camht), cv2.INTER_CUBIC)
            return img
        else:
            ret_val, img = self.cam.read()
            if img.shape[:2] != (self.camht, self.camwd):
                img = cv2.resize(img, (self.camwd, self.camht), cv2.INTER_CUBIC)
            return ret_val, img

    def retFace(self, img=None):
        if img is None:
            img = self.retCapture()
        if self.roi is None:
            self.roi = (0, 0, self.camwd, self.camht) 
        roiImg = img[int(self.roi[1]):int(self.roi[1]) + int(self.roi[3]),
                     int(self.roi[0]):int(self.roi[0]) + int(self.roi[2])]
        grayRoiImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = self.faceCascade.detectMultiScale(grayRoiImg, **self.detectargs)
        if len(faces) == 0:
            self.noDetectionCounter += 1
            if self.noDetectionCounter >= self.noDetectionLimit:
                return None
            return self.face1
        else:
            self.noDetectionCounter = 0
        for (x, y, w, h) in faces:
            if self.face1 is None:
                self.face1 = np.array([self.roi[0] + x, self.roi[1] + y, w, h])
                return self.face1
            else:
                if self.withinThreshold((x, y, w, h)):
                    return self.face1
                else:
                    self.face1 = np.array([self.roi[0] + x, self.roi[1] + y, w, h])
                    return self.face1
                
    

class BackgroundFormatter:
    def __init__(self, camwidth, camheight, rescale=8, mode='remove', img_path_alt='sample_12.jpeg', kernel_size=9, mask_kernel_size=11, smooth_iters=2, updation_iters=10):
        self.camwidth = camwidth
        self.camheight = camheight
        self.rescale = rescale
        self.mode = mode
        self.img_path_alt = img_path_alt
        self.bck_img = cv2.imread(img_path_alt)
        self.bck_img = cv2.resize(self.bck_img, (camwidth, camheight), cv2.INTER_CUBIC)
        self.updation_iters = updation_iters
        self.update_counter = -1
        self.sW = int(camwidth / rescale)
        self.sH = int(camheight / rescale)
        self.kernel_size = kernel_size
        self.mask_kernel_size = mask_kernel_size
        self.smooth_iters = smooth_iters
        self.back_blur_kernel = np.ones((kernel_size, kernel_size), np.float32) / (kernel_size**2)
        self.smooth_kernel = np.ones((mask_kernel_size, mask_kernel_size), np.float32) / (mask_kernel_size**2)
        self.backgoundModel = np.zeros((1, 65), np.float64)
        self.foregroundModel = np.zeros((1, 65), np.float64)
        self.masks = np.zeros((int(camwidth / rescale), int(camheight / rescale)))
        self.mask = self.masks

    def ret_mask(self, img, face_loc):
        self.update_counter += 1
        x, y, w, h = face_loc
        X, Y, W, H = [int(temp / self.rescale) for temp in (x, y, w, h)]
        if img.shape[:2] != (self.camheight, self.camwidth):
            print('WARNING::BackgroundFormatter::ret_mask(): The shapes of image do not match (Camera Width, Camera Height)')
        small_img = cv2.resize(img, (self.sW, self.sH), interpolation=cv2.INTER_LINEAR)
        rectangle = (max(1, X - int(W)), max(1, Y - int(H)), min(int(3 * W), self.sW), small_img.shape[0] - (Y - int(H)))
        if (self.update_counter % self.updation_iters == 0):
            self.masks, self.backgoundModel, self.foregroundModel = cv2.grabCut(small_img, self.masks, rectangle, self.backgoundModel, self.foregroundModel, 5, cv2.GC_INIT_WITH_RECT)
            self.mask = np.where((self.masks == 2) | (self.masks == 0), 0, 1).astype('uint8')
            self.mask = cv2.resize(self.mask, (self.camwidth, self.camheight), interpolation=cv2.INTER_LINEAR)
            for _ in range(self.smooth_iters):
                self.mask = cv2.filter2D(self.mask, -1, self.smooth_kernel)
            self.mask = self.mask[:, :, np.newaxis]
        return self.mask

    def apply_background(self, img, mask):
        if self.mode == None:
            return img
        elif self.mode == 'remove':
            return img * self.mask + self.bck_img * (1 - self.mask)
        elif self.mode == 'blur':
            alt_image = cv2.resize(img, (int(self.camwidth / 4), int(self.camheight / 4)))
            alt_image = cv2.filter2D(alt_image, -1, self.back_blur_kernel)
            alt_image = cv2.resize(alt_image, (self.camwidth, self.camheight))
            return img * self.mask + alt_image * (1 - self.mask)

if __name__ == '__main__':
    # Set the desired values directly here
    mode = "blur"  # Change this to the desired mode ('remove', 'blur', None)
    bg_path = "sample_12.jpeg"  # Change this to the desired background image path

    wid, hei = (640, 480)
    scale = 8
    update_iters = 10
    blur_kernel_size = 9
    mask_smooth_kernel_size = 15
    mask_smooth_iters = 5
    cap = FaceCapture(face_threshold=0.1, camht=hei, camwd=wid, rescale=scale)
    backformatter = BackgroundFormatter(camwidth=wid, camheight=hei, rescale=scale, 
                                        mode=mode, img_path_alt=bg_path, kernel_size=blur_kernel_size, mask_kernel_size=mask_smooth_kernel_size, 
                                        smooth_iters=mask_smooth_iters, updation_iters=update_iters)
    c = -1
    T = time()
    timer = 0
    timer_steps = 10
    while True:
        c += 1
        img = cap.retCapture()
        loc = cap.retFace(img)
        if loc is not None:
            mask = backformatter.ret_mask(img, loc)
            img = backformatter.apply_background(img, mask)
            cv2.imshow('camera', img)
        else:
            cv2.imshow('camera', img)
        if cv2.waitKey(1) == 27:
            break
        t = time()
        timer += t - T
        if c % timer_steps == 0:
            print("TIME: {} | FPS: {}".format(timer / timer_steps, timer))
            timer = 0
        T = t